# Inheco Incubator (Shaker)

| Summary | Photo |
|----------|--------|
| - [OEM Link](https://www.inheco.com/incubator-shaker.html)<br>- **Communication Protocol / Hardware**: Serial (FTDI)/ USB-A<br>- **Communication Level**: Firmware (documentation shared by OEM)<br>- Same command set for:<ul><li>Incubator "MP"</li><li>Incubator "DWP"</li><li>Incubator Shaker "MP"</li><li>Incubator Shaker "DWP"</li></ul>- Incubator Shaker "MP" VID:PID 0403:6001<br>- Takes in a single plate via a loading tray, heats it to set temperature, shakes it to set RPM | ![quadrants](img/inheco_incubator_shaker_mp_dwp.png) |


This notebook provides a minimal working example to connect to and interact with an INHECO Incubator Shaker over a USB serial connection. It demonstrates how to initialize communication, issue basic commands, and interpret the device’s responses.

## About the Device and Setup

INHECO incubator shakers are modular devices used for temperature control and agitation in lab automation setups. Multiple devices can be connected in a stack via USB, each assigned a **device ID** and **subdevice number** (usually 0–5). Commands are sent via a serial protocol and must be carefully constructed with CRC checks and timeouts.

Ensure:
- The correct **serial port** is selected (e.g., `/dev/cu.usbserial-140` on macOS).
- You know the **device ID** for the incubator you're communicating with.
- Only one command is sent at a time — the device cannot queue or process multiple requests concurrently.

## Setup Instructions (Physical)

| Summary                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | Photo                                            |
|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------|
| ![quadrants](img/inheco_incubator_shaker_physical_setup_overview.png) | |

## Setup Instructions (Programmatic)

In [1]:
import asyncio
import serial
import serial.tools.list_ports
import logging
from typing import Optional, List


class InhecoTask:
    """Represents a pending Inheco command awaiting a response."""
    def __init__(self, fut: asyncio.Future, cmd: str, timeout_time: float):
        self.fut = fut
        self.cmd = cmd
        self.timeout_time = timeout_time

class InhecoError(RuntimeError):
    """Represents a firmware-reported INHECO error."""
    def __init__(self, command: str, code: str, message: str):
        super().__init__(f"{command} failed with error {code}: {message}")
        self.command = command
        self.code = code
        self.message = message


ERROR_CODE_MAP = {
    "E00": "Unknown or generic error",
    "E01": "Drawer open",
    "E02": "Temperature sensor error",
    "E03": "Communication error",
    "E04": "Shaker not initialized",
    "E05": "Over-temperature",
    "E06": "Under-temperature",
    "E07": "No plate detected",
    "E08": "Plate jammed or not removable",
    "E09": "Hardware self-test failed",
    "E10": "Overcurrent on motor or fan",
    "E11": "EEPROM checksum error",
    "E12": "Power failure or voltage out of range",
    "E13": "Temperature deviation too high",
    "E14": "Safety sensor triggered",
    "E15": "Firmware internal error",
}



class InhecoIncubatorShakerBackend:
    """Backend for controlling an Inheco Incubator Shaker via RS-232 or auto-detected FTDI."""

    def __init__(
        self,
        port: Optional[str] = None,
        dip_switch_id: int = 2,
        stack_index: int = 0,
        write_timeout: float = 5.0,
        read_timeout: float = 10.0
    ):
        # --- Detect port automatically if not provided ---
        # if more than one Inheco Incubator Shaker (Stack) are to be used
        # you have to declare each ones port explicitly to avoid confusion
        VID = "0403"
        PID = "6001"

        if port is None:
            matching_ports = [
                p.device
                for p in serial.tools.list_ports.comports()
                if f"{VID}:{PID}" in p.hwid
            ]

            if not matching_ports:
                raise RuntimeError(f"No FTDI ({VID}:{PID}) serial devices found.")

            if len(matching_ports) > 1:
                ports_list = ", ".join(matching_ports)
                raise RuntimeError(
                    f"Multiple Inheco machines with VID:PID={VID}:{PID} found: "
                    f"{ports_list}. Please specify which port to use explicitly."
                )

            port = matching_ports[0]
        
        # Connection parameters
        self.port = port
        self.dip_switch_id = dip_switch_id
        self.stack_index = stack_index
        self.logger = logging.getLogger(__name__)

        # Serial connection
        self.ser = None
        self.write_timeout = write_timeout
        self.read_timeout = read_timeout

        # Cached machine status
        self.loading_tray = "unknown"
        self.incubator_type = "unknown"
        self.firmware_version = "unknown"

    # === Lifecycle ===

    async def setup(self):
        self.ser = serial.Serial(
            port=self.port,
            baudrate=19200,
            bytesize=serial.EIGHTBITS,
            parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE,
            timeout=0
        )
        self.ser.write_timeout = self.write_timeout

        self.firmware_version = await self.request_firmware_version()
        incubator_type = await self.request_incubator_type()
        serial_number = await self.request_serial_number()
        
        msg = (
            f"Connected to Inheco {incubator_type} on {self.port}\n"
            f"Machine serial number: {serial_number}\n"
            f"Firmware version: {self.firmware_version}"
        )
        print(msg)
        self.logger.info(msg)

        await self.initialize()
        await self.close()

    async def stop(self):
        if self.ser and self.ser.is_open:
            self.ser.close()
            self.logger.info("Disconnected from Inheco Incubator Shaker")

    # === Low-level I/O ===

    async def write(self, data: bytes):
        self.logger.debug(f"→ {data.hex()}")
        self.ser.write(data)

    async def _read_full_response(self, timeout: float = 5.0) -> bytes:
        """
        Read a full INHECO response frame.
        Stops on either:
          • <CR> (0x0D) terminator   — report replies
          • [header, 0x20, 0x60]     — ACK
          • [header, 0x28, 0x60]     — ERROR (long)
          • [header, 0x23, 0x60]     — ERROR (short, e.g. RER reply)
        """
        loop = asyncio.get_event_loop()
        start = loop.time()
        buffer = bytearray()
        header = 0xB0 + self.dip_switch_id
    
        tails = [
            bytes([header, 0x20, 0x60]),  # ACK
            bytes([header, 0x28, 0x60]),  # Error
            bytes([header, 0x23, 0x60]),  # Short error or RER
        ]
    
        while True:
            if self.ser.in_waiting:
                chunk = self.ser.read(self.ser.in_waiting)
                if chunk:
                    buffer.extend(chunk)
                    self.logger.debug(f"RX: {chunk.hex()}")
    
                    # Stop when a known tail or <CR> occurs
                    if b"\r" in buffer or any(buffer.endswith(t) for t in tails):
                        return bytes(buffer)
    
            if loop.time() - start > timeout:
                raise TimeoutError(f"Timed out waiting for complete response ({buffer.hex()})")
    
            await asyncio.sleep(0.01)



    # === Encoding / Decoding ===

    def _crc8(self, data: bytearray) -> int:
        crc = 0xA1
        for byte in data:
            d = byte
            for _ in range(8):
                if (d ^ crc) & 1:
                    crc ^= 0x18
                    crc >>= 1
                    crc |= 0x80
                else:
                    crc >>= 1
                d >>= 1
        return crc & 0xFF

    def _build_message(self, command: str, stack_index: int = 0) -> bytes:
        if not (0 <= stack_index <= 5):
            raise ValueError("stack_index must be between 0 and 5")
        full_command = f"T0{stack_index}{command}"
        cmd = full_command.encode("ascii")
        length = len(cmd) + 3
        address = 0x30 + self.dip_switch_id
        proto = 0xC0 + len(cmd)
        message = bytearray([length, address, proto]) + cmd
        crc = self._crc8(message)
        return bytes(message + bytearray([crc]))

    def _is_report_command(self, command: str) -> bool:
        return command.upper().startswith("R")

    def _parse_response_bytes(self, response) -> str:
        """Decode and clean INHECO response bytes."""
        expected_header = 0xB0 + self.dip_switch_id
        ack_tail = bytes([expected_header, 0x20, 0x60])
        err_tail = bytes([expected_header, 0x28, 0x60])
        short_err_tail = bytes([expected_header, 0x23, 0x60])
        short_tail = b"\r\n"
    
        if isinstance(response, str):
            cleaned = "".join(ch for ch in response if 32 <= ord(ch) <= 126)
            return cleaned.strip(" `\t\r\n")
    
        if not isinstance(response, (bytes, bytearray)):
            raise TypeError(f"Expected bytes or str, got {type(response).__name__}")
    
        if not response:
            raise ValueError("Empty response from machine.")
    
        # header check
        first = response[0]
        if not response.startswith(bytes([expected_header])) and not (
            65 <= first <= 90 or 97 <= first <= 122
        ):
            raise ValueError(f"Unexpected response header: {response[:1].hex()}")
    
        # known valid terminators
        valid_endings = (
            response.endswith(ack_tail)
            or response.endswith(err_tail)
            or response.endswith(short_err_tail)
            or response.endswith(short_tail)
            or response.endswith(b"\r")
        )
        if not valid_endings:
            raise ValueError(f"Unexpected response terminator: {response[-3:].hex()}")
    
        # short 0x23 tail → explicit “no details” indicator (per spec)
        if response.endswith(short_err_tail):
            return "E00"
    
        decoded = response.decode("ascii", errors="ignore")
        cleaned = "".join(ch for ch in decoded if 32 <= ord(ch) <= 126)
        return cleaned.rstrip(" `\t\r\n")


    def _validate_acknowledgement(self, response: bytes):
        expected_header = 0xB0 + self.dip_switch_id
        if not response.startswith(bytes([expected_header])):
            raise ValueError(f"Unexpected ack header: {response[:1].hex()}")
        if not response.endswith(bytes([expected_header, 0x20, 0x60])):
            raise ValueError(f"Unexpected ack terminator: {response[-3:].hex()}")

    def _is_error_response(self, response: bytes) -> bool:
        """Detects if response is an error frame."""
        header = 0xB0 + self.dip_switch_id
        return response.endswith(bytes([header, 0x28, 0x60]))

    
    # === High-level Command API ===

    async def send_command(
        self,
        command: str,
        delay: float = 0.2,
        read_timeout: Optional[float] = None
    ) -> str:
        msg = self._build_message(command, stack_index=self.stack_index)
        await self.write(msg)
        await asyncio.sleep(delay)
        response = await self._read_full_response(timeout=read_timeout or self.read_timeout)
    
        if not response:
            raise TimeoutError(f"No response from machine for command: {command}")
    
        # --- handle error frame ---
        if self._is_error_response(response):
            try:
                await asyncio.sleep(0.05)  # short grace delay for machine to prepare
                err_resp = await self.send_command("RER", delay=0.1)
                parsed = self._parse_response_bytes(err_resp) if err_resp else ""
                code = (parsed or "E00")[:3].upper()
        
                # --- Retry once if no code came back (some firmwares delay RER) ---
                if code == "E00":
                    await asyncio.sleep(0.1)
                    err_resp = await self.send_command("RER", delay=0.1)
                    parsed = self._parse_response_bytes(err_resp) if err_resp else ""
                    code = (parsed or "E00")[:3].upper()
        
                # --- Decode human-readable error ---
                if code == "E00":
                    message = (
                        "Generic error — machine did not provide detailed code "
                        "(firmware returned short [B0+ID 23 60] frame)"
                    )
                else:
                    message = ERROR_CODE_MAP.get(code, "Unknown error")
        
                raise InhecoError(command, code, message)
        
            except TimeoutError:
                raise InhecoError(command, "???", "Unknown error (RER query timed out)")
    
        # --- handle normal cases ---
        if self._is_report_command(command):
            return self._parse_response_bytes(response)
        else:
            self._validate_acknowledgement(response)
            return ""



    # === Public API ===

    # # # Inheco's "Report Commnads" # # # 
    async def request_firmware_version(self):
        """  """
        return await self.send_command("RFV0")

    async def request_serial_number(self):
        """ """
        return await self.send_command("RFV2")

    async def request_last_calibration_date(self):
        """
        Reports the date and an alphanumeric string (e.g. operator) of the last calibration for the machine.
        The Data is reported in the Format YYYY-MM-DD,xxxxx (Example: 2005-09-28,xxxxx).
        The five xs are alphanumeric wildcards.
        """
        resp = await incubator.send_command("RCM")
    
        return resp[:10]

    
    async def request_number_of_connected_machines(self):
        """ 
        How many Inheco Incubator (Shakers) are connected to the called machines?
        """
        
        resp = await self.send_command("RDA")
        # TODO: test on NOT stack_index == 0; 
        # Does it report the remainder of the above machines in the stack?
    
        return int(resp)

    async def request_labware_detection_threshold(self) -> int:
        """Report the labware detection threshold used during drawer movement (RDM)."""
        resp = await self.send_command("RDM")
        return int(resp)


    async def request_incubator_type(self):
        """
        Identify what machine functionality you have:
        Incubator (1) with shaker, (2) MP or DWP chamber 
        """
        
        incubator_type_dict = {
            "0": "incubator_mp", # no shaker!
            "1": "incubator_shaker_mp",
            "2": "incubator_dwp", # no shaker!
            "3": "incubator_shaker_dwp",
        }
        
        resp = await self.send_command("RTS")

        incubator_identified = incubator_type_dict[resp]
        
        self.incubator_type = incubator_identified
        
        return incubator_identified

    async def request_analog_values(self):
        """ Unknown feature """
        # resp = await self.send_command("RAV")
        return NotImplementedError("RAV not yet implemented, unknown feature.")

    async def request_plate_in_incubator(self):
        """ Request whether incubator currently hosts a plate """
        
        resp = await self.send_command("RLW")
    
        return resp == "1"

    async def request_analog_values(self):
        # resp = await self.send_command("RAV")
        return NotImplementedError("RAV not yet implemented, unknown feature.")

    async def request_operation_time_in_hours(self):
        """
        Report the overall operation time the machine has gone through in hours
        """
        resp = await self.send_command("RDC1")
        
        return int(resp)

    async def request_drawer_cyles_performed(self):
        """ Report the overall operation time in hours """
        resp = await self.send_command("RDC2")
        
        return int(resp)

    async def request_is_initialized(self):
        """ Request whether machine has undergone an initialization procedure """
        resp = await self.send_command("REE")
    
        return resp in {"0", "2"}

    async def request_plate_status_known(self):
        """
        Report whether the machine has knowledge of whether it hosts a plate or not.
        Note: use await incubator.request_plate_in_incubator() to *measure* whether 
          a plate is present
        """
        resp = await self.send_command("REE")
    
        return resp in {"0", "1"}



    
        
    # # # Inheco's "Set Commands" # # # 

    

    # # # Inheco's "Action Commands" # # # 
    async def initialize(self):
        return await self.send_command("AID")

    # Loading tray commands #
    async def open(self):
        await self.send_command("AOD")
        self.loading_tray = "open"

    async def close(self):
        await self.send_command("ACD")
        self.loading_tray = "closed"

    async def debug_rer(self):
        """Send RER manually and log raw bytes + parsed result."""
        print(">>> Sending RER manually...")
        msg = self._build_message("RER", stack_index=self.stack_index)
        await self.write(msg)
        await asyncio.sleep(0.1)
        response = await self._read_full_response(timeout=2.0)
        print(f"Raw RER response: {response.hex(' ')}")
        try:
            parsed = self._parse_response_bytes(response)
            print(f"Parsed: {parsed!r}")
        except Exception as e:
            print(f"Parse error: {e}")





In [2]:
# Autodetect FTDI device
incubator = InhecoIncubatorShakerBackend(
    dip_switch_id=2,
    stack_index=0
)
await incubator.setup()

Connected to Inheco incubator_shaker_mp on /dev/cu.usbserial-130
Machine serial number: 2013
Firmware version: IncShak_C_V3.50_04/2012


In [3]:
await incubator.send_command("REE")

'0'

In [4]:
await incubator.request_plate_in_incubator()

False

In [7]:
await incubator.debug_rer()

>>> Sending RER manually...
Raw RER response: b2 23 60
Parsed: 'E00'


In [7]:
await incubator.close()

In [5]:
incubator.loading_tray

await incubator.request_operation_time_in_hours()

50

In [6]:
await incubator.request_is_initialized()

True

In [6]:
 await incubator.send_command("RLW")

'0'

## Usage

### Testing & Investigation

'2'

In [6]:
await incubator.send_command("REE")

'0'

In [13]:
await incubator.send_command("RDC2")

# Reports the status of the initialisation of the device and the status of the Lab-Ware detection
# if
# 0 Device initialised
# 1 Device not initialised
# 2 Lab-Ware status unknown
# 3 Device not initialised and Lab-Ware status unknow

'1675'

'1'

In [16]:
for x in [
    0, 1, 2, 3, 
    5, 7, 10, 15,
    20, 21, 22, 23,
    24, 25
]:
    print(x, await incubator.send_command(f"RFV{x}") )

# Reports the date and an alphanumeric string (e.g. operator) of the last calibration for the device.
# The Data is reported in the Format YYYY-MM-DD,xxxxx (Example: 2005-09-28,xxxxx).
# The five xs are alphanumeric wildcards.

0 IncShak_C_V3.50_04/2012
1 BOOT_C_V3.12_05/2007
2 2013
3 0
5 COPYRIGHT INHECO
7 1
10 IncShak
15 350
20 325
21 315
22 IncShak_R_V3.25_12/2011
23 BOOT_R_V1.10_01/2007
24 IncShak_S_V3.15_05/2011
25 BOOT_S_V1.10_01/2007


'incubator_shaker_mp'

In [9]:
await incubator.stop()

In [15]:

def request_machine_initialisation_status():
    """ """
    resp = await incubator.send_command("REE")

# Reports the status of the initialisation of the device and the status of the Lab-Ware detection
# if
# 0 Device initialised
# 1 Device not initialised
# 2 Lab-Ware status unknown
# 3 Device not initialised and Lab-Ware status unknow

'0'

### Loading Drawer

In [14]:
await incubator.open_lid()
# await asyncio.sleep(5)
# await incubator.close_lid()

In [16]:
await incubator.close_lid()

### Temperature Control

In [8]:
for t in range(5):
    temp_meas = {}
    
    for x in range(1,4):
        temp_meas[x] = await incubator.send_command(f"RAT{x}", read_timeout=60)

    print(t, temp_meas)
    time.sleep(1)

# This command reports the actual ambient or slot temperatures of the three sensors.
# It can be chosen whether the evaluated temperature or the sensor value should be reported.
# The temperatures are reported in 1/10 °C: 345 = 34,5 °C

0 {1: '172', 2: '173', 3: '173'}
1 {1: '172', 2: '173', 3: '173'}
2 {1: '172', 2: '173', 3: '173'}
3 {1: '173', 2: '173', 3: '173'}
4 {1: '172', 2: '173', 3: '173'}


In [ ]:
await incubator.send_command("RTT", read_timeout=60)


In [ ]:
await incubator.send_command("STT700", read_timeout=60)


### Shaking Control

TODO

In [4]:
import serial.tools.list_ports

def list_serial_devices():
    ports = serial.tools.list_ports.comports()
    for port in ports:
        print(f"Device: {port.device}")
        print(f"  Description: {port.description}")
        print(f"  HWID: {port.hwid}")
        print("-" * 40)

if __name__ == "__main__":
    print("Scanning USB serial devices...")
    list_serial_devices()


Scanning USB serial devices...
Device: /dev/cu.debug-console
  Description: n/a
  HWID: n/a
----------------------------------------
Device: /dev/cu.BoseQC35II
  Description: n/a
  HWID: n/a
----------------------------------------
Device: /dev/cu.Bluetooth-Incoming-Port
  Description: n/a
  HWID: n/a
----------------------------------------
Device: /dev/cu.usbserial-140
  Description: USB <-> Serial
  HWID: USB VID:PID=0403:6001 LOCATION=0-1.4
----------------------------------------


### Closing Connection

In [5]:
await incubator.stop()